# MALTbot — Daily Colab Batch Runner (canonical)

## 원하는 UX (사용자 기준)
1) Colab에서 `notebooks/MALTbot_2.ipynb` 열기
2) 맨 위 **CONFIG 셀만** 수정
   - `BATCH_RUN_NAME`: 예) `2026-02-21_batch1`
   - `EXPERIMENTS`: 예) `["baseline_chgnet", "chgnet_seed43", "chgnet_lr_schedule", ...]`
   - (옵션) `GH_PUSH=True/False`
3) **Run all**

실행 후 자동으로:
- `results/daily/<DATE>/<BATCH_RUN_NAME>/<exp_name>/results.json` 생성 (실험별 1개)
- `RESULTS.md`에 실험별 1줄 append
- GitHub에 브랜치 `colab-<DATE>-<SAFE_BATCH>`로 한 번에 push + PR 링크 출력

> 원칙: `main`에 직접 push하지 않고, PR로만 반영.


In [1]:
# [FIX] Force reinstall torch to resolve Colab Python 3.12 circular import error
!pip install --upgrade torch --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [2]:

# --- Quick UX ---
# 아래 변수만 바꾸고 Run all:
#   - BATCH_RUN_NAME: 예) 2026-02-21_batch1
#   - EXPERIMENTS: 예) ["baseline_chgnet", "chgnet_seed43", ...]
#   - GH_PUSH: True면 자동 push + PR 링크 출력
# 실행 후 자동으로:
#   - results/daily/<DATE>/<SAFE_BATCH>/<exp_name>/results.json 생성
#   - RESULTS.md에 실험별 1줄 append
#   - GitHub 브랜치 colab-<DATE>-<SAFE_BATCH>로 push
# ----------------

# CONFIG (edit only this cell)
from datetime import datetime
from zoneinfo import ZoneInfo
import os, re

DATE = datetime.now(ZoneInfo("Asia/Seoul")).strftime("%Y-%m-%d")
BATCH_RUN_NAME = "daily10"
TASK = "matbench_mp_e_form"
EXPERIMENTS = [
    "chgnet_pretrained_infer",
    "chgnet_head_finetune_freeze",
    "chgnet_full_finetune",
]
GH_PUSH = True

SAFE_BATCH = re.sub(r"[^A-Za-z0-9._-]+", "-", BATCH_RUN_NAME).strip("-") or "batch"

os.environ["MALTBOT_DATE"] = DATE
os.environ["MALTBOT_BATCH_RUN_NAME"] = SAFE_BATCH
os.environ["MALTBOT_TASK"] = TASK
os.environ["MALTBOT_EXPERIMENTS"] = ",".join(EXPERIMENTS)
os.environ["MALTBOT_GH_PUSH"] = "1" if GH_PUSH else "0"

print({
    "DATE": DATE,
    "BATCH_RUN_NAME(raw)": BATCH_RUN_NAME,
    "SAFE_BATCH": SAFE_BATCH,
    "TASK": TASK,
    "EXPERIMENTS": EXPERIMENTS,
    "GH_PUSH": GH_PUSH,
})


{'DATE': '2026-02-23', 'BATCH_RUN_NAME(raw)': 'daily10', 'SAFE_BATCH': 'daily10', 'TASK': 'matbench_mp_e_form', 'EXPERIMENTS': ['chgnet_pretrained_infer', 'chgnet_head_finetune_freeze', 'chgnet_full_finetune', 'chgnet_ensemble3', 'mlp_pretrained_infer_fallback', 'mlp_head_finetune_freeze', 'chgnet_seed43', 'chgnet_seed44', 'chgnet_lr_schedule', 'chgnet_target_transform', 'chgnet_ema', 'chgnet_epochs80_seed43'], 'GH_PUSH': True}


In [3]:

import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


Python: 3.12.12
Torch: 2.10.0+cu128
CUDA available: True
GPU: NVIDIA L4


In [4]:

%%bash
set -euo pipefail
REPO_DIR="/content/MALTbot"
REPO_URL="https://github.com/seanwoory/MALTbot.git"
REQUIRED_COMMIT="13afb95"

if [ -d "${REPO_DIR}/.git" ]; then
  echo "[info] Repo exists. Pull latest main..."
  cd "${REPO_DIR}"
  git fetch origin
  git checkout main || true
  git pull --ff-only origin main
else
  echo "[info] Cloning repo..."
  git clone "${REPO_URL}" "${REPO_DIR}"
  cd "${REPO_DIR}"
fi

echo "[info] HEAD full commit: $(git rev-parse HEAD)"
echo "[info] HEAD short commit: $(git rev-parse --short HEAD)"

if ! git merge-base --is-ancestor "${REQUIRED_COMMIT}" HEAD; then
  echo "[error] Repo is older than required commit ${REQUIRED_COMMIT}. Please re-run this cell."
  exit 1
fi


[info] Cloning repo...
f422ccf


Cloning into '/content/MALTbot'...


In [5]:

%%bash
set +e
LOG=/tmp/maltbot_pip_install.log
MARKER=/tmp/maltbot_install_failed
rm -f "$MARKER"

python3 -m pip install -U pip setuptools wheel >"$LOG" 2>&1
S1=$?
python3 -m pip install   "numpy<2"   scipy   pandas   scikit-learn   pymatgen   pyyaml   tqdm >>"$LOG" 2>&1
S2=$?

# matbench: pip first, fallback to official GitHub
python3 -m pip install matbench >>"$LOG" 2>&1
S3=$?

# CHGNet for structure-based runs
python3 -m pip install chgnet >>"$LOG" 2>&1
S4=$?
if [ $S3 -ne 0 ]; then
  python3 -m pip install "git+https://github.com/materialsproject/matbench.git" >>"$LOG" 2>&1
  S3=$?
fi

if [ $S1 -ne 0 ] || [ $S2 -ne 0 ] || [ $S3 -ne 0 ] || [ $S4 -ne 0 ]; then
  echo "[warn] Dependency install had failures. See $LOG"
  touch "$MARKER"
else
  echo "[ok] Dependency install completed"
fi

# never install torch in Colab; use preinstalled torch
python3 - <<'PY2'
import torch, platform
print('Python:', platform.python_version())
print('Torch (preinstalled):', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
PY2

# keep notebook progressing; debug cell will print details if failed
exit 0


Process is terminated.


In [6]:

from pathlib import Path
import subprocess, sys, platform

marker = Path('/tmp/maltbot_install_failed')
if marker.exists():
    print('[debug] Install failure detected. Environment snapshot:')
    print('Python executable:', sys.executable)
    print('Python version:', platform.python_version())
    print()
    print('[pip freeze top 30]')
    out = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'], text=True, errors='ignore')
    lines = [ln for ln in out.splitlines() if ln.strip()]
    for ln in lines[:30]:
        print(ln)
    print()
    print('[tip] Full install log: /tmp/maltbot_pip_install.log')
else:
    print('[ok] Install debug skipped (no failure marker).')


[ok] Install debug skipped (no failure marker).


In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)


MessageError: Error: credential propagation was unsuccessful

In [ ]:

from google.colab import userdata
from getpass import getpass
import os

# Load token before experiment loop to support per-experiment real-time push
if os.environ.get("MALTBOT_GH_PUSH") == "1":
    token = None
    try:
        raw = userdata.get("GH_TOKEN")
        token = raw if isinstance(raw, str) else None
    except Exception:
        token = None

    if not token or not token.strip():
        token = getpass("Paste GH_TOKEN for per-experiment push (input hidden): ")

    token = token.strip() if isinstance(token, str) else ""
    if not token:
        raise ValueError("GH_TOKEN is required when GH_PUSH=True")

    os.environ["GH_TOKEN"] = token
    print("GH_TOKEN loaded early for per-experiment push (hidden)")
else:
    print("GH_PUSH=False; skip early token load")


In [ ]:

import os, subprocess, sys

repo = "/content/MALTbot"
experiments = [x.strip() for x in os.environ.get("MALTBOT_EXPERIMENTS", "").split(",") if x.strip()]
if not experiments:
    raise ValueError("EXPERIMENTS is empty")

gh_push = os.environ.get("MALTBOT_GH_PUSH") == "1"
date = os.environ.get("MALTBOT_DATE")
batch = os.environ.get("MALTBOT_BATCH_RUN_NAME")
branch = f"colab-{date}-{batch}" if date and batch else None

if gh_push:
    token = os.environ.get("GH_TOKEN", "")
    if not token:
        raise ValueError("GH_PUSH=True but GH_TOKEN is not set")
    subprocess.run(["git", "checkout", "-B", branch], cwd=repo, check=False)
    subprocess.run(["git", "config", "user.name", "colab-bot"], cwd=repo, check=True)
    subprocess.run(["git", "config", "user.email", "colab-bot@users.noreply.github.com"], cwd=repo, check=True)
    subprocess.run(["git", "remote", "set-url", "origin", f"https://{token}@github.com/seanwoory/MALTbot.git"], cwd=repo, check=True)

for exp in experiments:
    print(f"\n===== RUN {exp} =====")
    cmd = [sys.executable, "scripts/run_experiment.py", "--exp-name", exp]

    # Real-time streaming output + buffer capture
    proc = subprocess.Popen(
        cmd,
        cwd=repo,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
    )

    out_lines = []
    assert proc.stdout is not None
    for line in proc.stdout:
        print(line, end="")
        out_lines.append(line)

    return_code = proc.wait()
    full_output = "".join(out_lines)
    print(f"status code: {return_code}")

    # Optional: quick parse hints for debugging visibility
    wrote_line = None
    metric_line = None
    for ln in out_lines:
        if "Wrote results to" in ln:
            wrote_line = ln.strip()
        if "FINAL_METRIC_MAE=" in ln:
            metric_line = ln.strip()
    if wrote_line:
        print(f"[info] {wrote_line}")
    if metric_line:
        print(f"[info] {metric_line}")

    # Real-time reporting: commit/push after each experiment run
    if gh_push and branch:
        exp_path = f"results/daily/{date}/{batch}/{exp}/results.json"
        subprocess.run(["git", "add", exp_path, "RESULTS.md"], cwd=repo, check=False)
        diff = subprocess.run(["git", "diff", "--cached", "--quiet"], cwd=repo)
        if diff.returncode != 0:
            msg = f"results: {exp}"
            subprocess.run(["git", "commit", "-m", msg], cwd=repo, check=False)
            subprocess.run(["git", "push", "-u", "origin", branch], cwd=repo, check=False)
            print(f"[pushed] {exp} -> {branch}")
        else:
            print(f"[no changes] {exp}")

print("Batch finished")


In [ ]:

%%bash
set -euo pipefail
cd /content/MALTbot
find "results/daily/${MALTBOT_DATE}/${MALTBOT_BATCH_RUN_NAME}" -type f -name "results.json" | sort || true
tail -n 20 RESULTS.md


In [ ]:

from google.colab import userdata
from getpass import getpass
import os

if os.environ.get("MALTBOT_GH_PUSH") != "1":
    print("GH_PUSH=False; skip token load")
else:
    token = None
    try:
        raw = userdata.get("GH_TOKEN")
        token = raw if isinstance(raw, str) else None
    except Exception:
        token = None

    if not token or not token.strip():
        token = getpass("Paste GH_TOKEN (input hidden): ")

    token = token.strip() if isinstance(token, str) else ""
    if not token:
        raise ValueError("GH_TOKEN is required when GH_PUSH=True")

    os.environ["GH_TOKEN"] = token
    print("GH_TOKEN loaded (hidden)")


In [ ]:

%%bash
set -euo pipefail
cd /content/MALTbot

if [ "${MALTBOT_GH_PUSH}" != "1" ]; then
  echo "GH_PUSH=False; skip preflight"
  exit 0
fi

: "${GH_TOKEN:?GH_TOKEN is not set}"
: "${MALTBOT_DATE:?MALTBOT_DATE missing}"
: "${MALTBOT_BATCH_RUN_NAME:?MALTBOT_BATCH_RUN_NAME missing}"

BRANCH="colab-${MALTBOT_DATE}-${MALTBOT_BATCH_RUN_NAME}"

git checkout -B "${BRANCH}" >/dev/null 2>&1 || git checkout "${BRANCH}" >/dev/null 2>&1

git remote set-url origin "https://${GH_TOKEN}@github.com/seanwoory/MALTbot.git"

echo "Preflight OK: token set, origin updated, branch=${BRANCH}"


In [ ]:

%%bash
set -euo pipefail
cd /content/MALTbot

if [ "${MALTBOT_GH_PUSH}" != "1" ]; then
  echo "GH_PUSH=False; skip git push"
  exit 0
fi

: "${GH_TOKEN:?GH_TOKEN is not set}"
: "${MALTBOT_DATE:?MALTBOT_DATE missing}"
: "${MALTBOT_BATCH_RUN_NAME:?MALTBOT_BATCH_RUN_NAME missing}"

BRANCH="colab-${MALTBOT_DATE}-${MALTBOT_BATCH_RUN_NAME}"

git config user.name "colab-bot"
git config user.email "colab-bot@users.noreply.github.com"

git checkout -B "${BRANCH}"

git remote set-url origin "https://${GH_TOKEN}@github.com/seanwoory/MALTbot.git"

git add "results/daily/${MALTBOT_DATE}/${MALTBOT_BATCH_RUN_NAME}" RESULTS.md

if git diff --cached --quiet; then
  echo "No staged changes"
else
  git commit -m "results: ${MALTBOT_DATE} ${MALTBOT_BATCH_RUN_NAME}"
  git config pull.rebase true
  git pull origin "${BRANCH}" || true
  git push -u origin "${BRANCH}"
fi

echo "Create PR: https://github.com/seanwoory/MALTbot/compare/main...${BRANCH}?expand=1"
